---
title: "Sample Size for A/B Testing"
author: "chris"
date: 2024-03-31
draft: false
---

outline:

- work convo that led to reviving old project
- a/b testing layman terms
- examples; rate of ... conversions, approvals, etc
- number of binary outcomes -> binomial -> normal approximation
- why sample size is important and minimum detectable effect
- derivation (with figures/graphics) and show calculator

$$X \sim B(n, p)$$

where $n$ is number of trials and $p$ is the probability of a success

the expected value $E[X] = np$ and variance $Var(X) = np(1 - p)$